In [ ]:
# Step 1
import data
data.download_arxiv(save_path)
papers_parsed_dict = data.process_arxiv(save_path, YEARS)
# Step 2
import preprocessing, analyze
grouped_work, edge_df, author_df = preprocessing.preprocessing_for_embedding(save_path, '2000', 'arxiv')
path=f'{save_path}/2000/arxiv/'
data_object = analyze.embed_and_save(path,'full', 'tfidf', grouped_work, edge_df, author_df)
logging.info(f"Finished embedding for {path}")
analyze.calc_save_network_statistics(path,'full', data_object)
# Step 3
import preprocessing
path=f'{save_path}/2000/arxiv/full'
preprocessing.preprocessing_for_model(path,'tfidf','gat')
# Step 4
# ZEROSHOT
import torch
save_path='/ix/djishnu/Common_Folder/Coauth_Rev/finale2'
year, data, component, embedding, model, control = '2000', 'arxiv', 'full', 'tfidf', 'zeroshot', None
batch_size = 2048
neg_ratio = 1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
path = f"{save_path}/{year}/{data}/{component}/{embedding}/{model}"
if control is not None:
    path = f"{path}/{control}"

if 'zeroshot' not in model:
        logging.error("This scripts only supports 'zeroshot'. 'zeroshot' not found in models. Hence skipping.")
        exit()

import analyze, utils
all_data, _, _, test_data = utils._read_data(path,device)
analyze.run_zeroshot(path, all_data,test_data,batch_size,neg_ratio,device)
# Step 4
# GAT
from torch_geometric.nn import GAT
import torch
class GAT_Module(torch.nn.Module):
    def __init__(self, out_channels = 256, hidden_layers = 2, v2=True, concat=True,heads=1,add_self_loops=False):
        super(GAT_Module,self).__init__()
        self.gat = GAT(in_channels = -1, hidden_channels = 2*out_channels, num_layers = hidden_layers, out_channnels=out_channels, concat=concat, heads=heads, add_self_loops=add_self_loops)

    def forward(self, data):
        x = data.x.to(torch.float32)
        z = self.gat(x,data.edge_index)
        return z

save_path='/ix/djishnu/Common_Folder/Coauth_Rev/finale2'
year, data, component, embedding, model, control = '2000', 'arxiv', 'full', 'tfidf', 'gat', None
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
path = f"{save_path}/{year}/{data}/{component}/{embedding}/{model}"
if control is not None:
    path = f"{path}/{control}"

if model != 'gat' and model != 'gat_graph_embed':
    logging.error("This script only supports 'gat' or 'gat_graph_embed'. 'gat' or 'gat_graph_embed' not found in models. Hence Exiting.")
    exit()

if model == 'gat_graph_embed':
    try:
        assert (-1 in all_data.y) and (len(np.where((all_data.y==-1))[0])==1)
        graph_ed=True
        neg_node = np.where((all_data.y==-1))[0][0] # Pseudo node
    except:
        logging.error(f'model = {model} but -1 not found in all_data.y or more than 1 pseudo node found. Exiting...')
        exit()
out_channels = 256
hidden_layers = 2
batch_size = 2048
neg_ratio = 1
epochs = 4
batch_size = 2048
neg_ratio = 1
log_epoch = 1

gat = GAT_Module(out_channels,hidden_layers).to(device)
optimizer = torch.optim.Adam(gat.parameters(), lr=0.01, weight_decay=0.001)
graph_ed = False
neg_node = None

import analyze, utils
all_data, train_data, val_data, test_data = utils._read_data(path,device)
analyze.run_gat(path, all_data, train_data, val_data, test_data, batch_size, neg_ratio, device, gat, optimizer, epochs, log_epoch, neg_node, graph_ed)          

In [2]:
YEARS = ['2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','2023']
datasets = ['arxiv', 'openalex']
graph_components = ['full','lcc']
embedding_modes = ['tfidf','bert']
models = ['zeroshot','gat','gat_graph_embed']
controls = ['shuffle_y', 'shuffle_x']
restart_level = None
save_path='/ix/djishnu/Common_Folder/Coauth_Rev/finale3'
save_per_year = True
# process_restart(restart_level)

In [4]:
import dask, pandas as pd, pickle, os, logging, itertools, argparse,ast
from dask.distributed import Client, as_completed
from dask_jobqueue import SLURMCluster
from tqdm import tqdm
from collections import defaultdict
from plotter import tokenize_title_per_year, process_year

# Creating directories for the the analysis
os.makedirs(f"{save_path}/slurm_outs/5_data_combining_dask", exist_ok=True,mode=0o775)
if not os.path.exists(f'{save_path}/combined_year_results/tfidf_features'):
    os.makedirs(f'{save_path}/combined_year_results/tfidf_features', exist_ok=False, mode=0o755)

if not os.path.exists(f'{save_path}/combined_year_results/auc_aupr'):
    os.makedirs(f'{save_path}/combined_year_results/auc_aupr', exist_ok=False, mode=0o755)

# Setting dask client
logging.info("Starting dask client...")
cluster = SLURMCluster(
        cores=1,
        memory='10GB',  # Memory per process
        walltime='0-01:00:00',
        account='djishnu',
        job_extra_directives=[  '--job-name=data_combining',
                                '--cluster=htc',
                                f'--output={save_path}/slurm_outs/5_data_combining_dask/%x_%A_%a.out',
                            ]
    )
cluster.adapt(minimum=1, maximum=128)
client = Client(cluster)

data= datasets[0]
# country = 'ALL'

# Reading the countries from the file
countries=[]
cntry_file_path = '/ix/djishnu/Swapnil/coauthorship/Co-Authorship/inputs/countries.txt'
with open(f'{cntry_file_path}', 'r') as file:
    for line in file:
        stripped_line = line.strip()
        if not line.startswith('#') and len(stripped_line) > 0:
            countries.append(stripped_line)

# Preparing arguments for AUC_AUPR and TFIDF
arguments = list(itertools.product(YEARS, [data], graph_components, embedding_modes, models))
if controls is not None:
    arguments = arguments + list(itertools.product(YEARS, [data], graph_components, embedding_modes, models, controls))

if (data =='openalex'):
    args_for_map_auc = list(itertools.product([save_path],arguments, countries))
    args_for_map_tfidf = list(itertools.product([save_path], YEARS, [data], countries, [save_per_year]))
elif (data == 'arxiv'):
    args_for_map_auc = list(itertools.product([save_path],arguments, ['ALL']))
    args_for_map_tfidf = list(itertools.product([save_path], YEARS, [data], ['ALL'], [save_per_year]))
else:
    logging.error(f"Country {country} not found in the {data}. Skipping...")
    client.close()
    cluster.close()
    exit()
# Preparing variables for AUC_AUPR
all_model_result, all_model_quad_result = [], []
all_columns_labels = ['arguments','country', 'AUC','AUPR']
quad_columns_labels = ['arguments', 'country', 'quadrant_logic', 'Q1_AuC', 'Q2_AuC', 'Q3_AuC', 'Q4_AuC', 'Q1_AuPR', 'Q2_AuPR', 'Q3_AuPR', 'Q4_AuPR']
# Preparing variables for TFIDF
tfidf_features = defaultdict(float)
# Submit futures with tags and store them in a dictionary
futures_auc = {client.submit(process_year, *args): 'auc' for args in args_for_map_auc}
futures_tfidf = {client.submit(tokenize_title_per_year, *args): 'tfidf' for args in args_for_map_tfidf}

# Combine the futures into a single dictionary
all_futures = {**futures_auc, **futures_tfidf}

# Collect results and exceptions with tags
results_auc, results_tfidf, exceptions = [], [], []

for future in as_completed(all_futures):
    tag = all_futures[future]
    try:
        result = future.result()
        if tag == 'auc':
            results_auc.append(result)
        else:
            results_tfidf.append(result)
    except Exception as e:
        exceptions.append((future, e))

#Combining auc_aupr features across years
for i,result in enumerate(results_auc):
    if (result[0] is not None) and (result[1] is not None) and (result[2] is not None):   
        all_model_result.append(result[0])
        all_model_quad_result.append(result[1])
        all_model_quad_result.append(result[2])
    else:
        continue
all_model_result = [entry if isinstance(entry, (list, tuple)) else [entry] for entry in all_model_result] #Dealing with NaNs
all_model_quad_result = [entry if isinstance(entry, (list, tuple)) else [entry] for entry in all_model_quad_result] #Dealing with NaNs
pd.DataFrame(all_model_result, columns=all_columns_labels).to_csv(f'{save_path}/combined_year_results/auc_aupr/{data}_compiled_results.csv',index=False)
pd.DataFrame(all_model_quad_result, columns=quad_columns_labels).to_csv(f'{save_path}/combined_year_results/auc_aupr/{data}_compiled_quad_results.csv',index=False)
logging.info(f"Completed AUC AUPR for {data}")

#Combining tfidf features across years
for result in results_tfidf:
    if result is not None:
        for key, value in result.items():
            tfidf_features[key] += value
pickle.dump(tfidf_features, open(f'{save_path}/combined_year_results/tfidf_features/{data}_tfidf_combined_across_years.pkl', 'wb'))
logging.info(f"Completed embedding for {data}")
# Shut down the cluster
client.close()
cluster.close()

/ix/djishnu/Swapnil/.conda/envs/coauth_env/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34323 instead
  warnings.warn(


In [20]:
# Shut down the cluster
client.close()
cluster.close()

In [ ]:
data= datasets[0]
if not os.path.exists(f'{save_path}/combined_year_results/tfidf_features'):
    os.makedirs(f'{save_path}/combined_year_results/tfidf_features', exist_ok=False, mode=0o755)

if not os.path.exists(f'{save_path}/combined_year_results/auc_aupr'):
    os.makedirs(f'{save_path}/combined_year_results/auc_aupr', exist_ok=False, mode=0o755)
arguments = list(itertools.product(YEARS, [data], graph_components, embedding_modes, models))
countries = ['ALL']
for country in countries:
        if (data =='openalex') or (data == 'arxiv' and country == 'ALL'):
            try:
                calc_auc_aupr_across_years(save_path, data, arguments, country)
                print(f"Finished calculating AUC and AUPR for country {country} in {data} data.", flush=True)
            except Exception as e:
                logging.error(f"Error in calculating AUC and AUPR for country {country} in {data} data. Error: {e}")
                continue
            
            # with mp.Pool(processes=mp.cpu_count()) as pool:
            #     results = pool.starmap(tokenize_title_per_year, [(save_path, year, data, country,save_per_year) for year in YEARS])
            
            # #Combining tfidf features across years
            # tfidf_features = defaultdict(float) 
            # for result in results:
            #     for key, value in result.items():
            #         tfidf_features[key] += value
            # pickle.dump(tfidf_features, open(f'{save_path}/combined_year_results/tfidf_features/{country}_{data}_tfidf_combined_across_years.pkl', 'wb'))
        else:
            logging.error(f"Country {country} not found in the {data}. Skipping...")
            continue


In [ ]:
import plotter, preprocessing,pickle,numpy as np
from utils import _LemmaTokenizer, _HexCodeFromFrequencyDict
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.pipeline import Pipeline
from collections import defaultdict
for dataset in datasets:
    for year in YEARS:
        path=f'{save_path}/{year}/{dataset}/'
        tfidf_features = defaultdict(float)
        def tokenize_title_across_years(paper_titles,tfidf_features):
            lemma_tokenizer=_LemmaTokenizer()
            stop_words = set(stopwords.words('english')) 
            token_stop = lemma_tokenizer(' '.join(stop_words))
            pipe = Pipeline([('count', CountVectorizer(tokenizer=lemma_tokenizer, stop_words=token_stop, max_features=768)), ('tfid', TfidfTransformer())])
            pipe.fit(paper_titles)
            cts = pipe['count'].transform(paper_titles)
            for key, value in zip(pipe.get_feature_names_out(), np.sum(cts,axis=0).A1 / (1.0*len(paper_titles))):
                tfidf_features[key]+=(value)
            return tfidf_features

        with open(f'{path}/paper_titles.pkl', 'rb') as f:
            paper_titles = pickle.load(f)
        tfidf_features = tokenize_title_across_years(paper_titles, tfidf_features)
    
    wc = wordcloud.WordCloud(background_color='white',
                         color_func=_HexCodeFromFrequencyDict(tfidf_features), 
                         height=400)
    wc.generate_from_frequencies(tfidf_features)
    with open(f'{save_path}/combined_files/{dataset}_wc_all_yrs.svg',"w") as f:
        f.write(wc.to_svg(embed_font=True))